# Clean Worksheet

### Cleaning the dataset to use in later analysis.

---

In [64]:
# Importing pandas as uploading the data.
import pandas as pd
df = pd.read_excel(r"C:\Users\Windows 11\OneDrive\Desktop\Projects\Volunteer_Project\sample_dataset.xlsx")
df.head()

,Product,Supplier,Date,Price
0,Pipes,Skyline Traders,2023-06-03 10:32:53,294.85
1,Wood,IronHouse,2023-08-08 06:00:48,272.98
2,Wood,GreenField Ltd,2023-05-31 03:00:37,47.59
3,Bricks,GoldenHammer,2023-09-15 03:40:02,33.23
4,Wood,IronHouse,2023-04-12 23:42:14,57.21


In [65]:
# Examining the data.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1003 entries, 0 to 1002
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Product   1002 non-null   object        
 1   Supplier  1002 non-null   object        
 2   Date      1002 non-null   datetime64[ns]
 3   Price     1002 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 31.5+ KB


In [66]:
# Examining the numeric columns to make sure there are no anomalies.
df.describe()

,Date,Price
count,1002,1002.000000
mean,2023-07-02 21:49:12.499002112,249.909142
min,2023-01-01 00:18:58,5.110000
25%,2023-04-06 15:59:42.750000128,127.205000
50%,2023-07-02 00:28:22.500000,246.865000
75%,2023-09-28 03:16:59,372.427500
max,2024-01-01 22:45:56,498.890000
std,NaN,141.770590


In [67]:
# Examining the categorical columns to understand the data better.
categorical_columns = df.dtypes[df.dtypes == "object"].index
categorical_data = df[categorical_columns]
categorical_data.describe()

,Product,Supplier
count,1002,1002
unique,10,10
top,Pipes,GreenField Ltd
freq,113,117


In [68]:
# Spotting and counting duplicates.
df.isnull().sum()

Product     1
Supplier    1
Date        1
Price       1
dtype: int64

In [69]:
# Spotting and counting missing values.
df.isna().sum()

Product     1
Supplier    1
Date        1
Price       1
dtype: int64

In [70]:
# Deleting the duplicates and the rows that contain missing values.
# There were just a few missing values, that's why by deleting them I won't loose any valuable information.
df.dropna(inplace = True)
df.drop_duplicates(inplace = True)

In [71]:
# 8 rows were deleted.
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 996 entries, 0 to 999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Product   996 non-null    object        
 1   Supplier  996 non-null    object        
 2   Date      996 non-null    datetime64[ns]
 3   Price     996 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 38.9+ KB


In [72]:
# Cleaning the Product column.
df["Product"] = (
    df["Product"]
    .astype(str)                                      # making sure the type is string
    .str.replace(r"\s+", " ", regex=True)             # removing white spcaes
    .str.replace(r"\u00A0", " ", regex=True)          # replacing non-breaking spaces
    .str.replace(r"[\u200B-\u200D]", "", regex=True)  # removing zero-width spaces
    .str.replace(",", "", regex=False)                # removing commas
    .str.strip()                                      # stripping
    .str.upper()                                      # making everything uppercase
)

In [73]:
# Cleaning the Supplier column.
df["Supplier"] = (
    df["Supplier"]
    .astype(str)                                      # making sure the type is string
    .str.replace(r"\s+", " ", regex=True)             # removing white spcaes
    .str.replace(r"\u00A0", " ", regex=True)          # replacing non-breaking spaces
    .str.replace(r"[\u200B-\u200D]", "", regex=True)  # removing zero-width spaces
    .str.replace(",", "", regex=False)                # removing commas
    .str.strip()                                      # stripping
    .str.upper()                                      # making everything uppercase
)

In [74]:
df.head()

,Product,Supplier,Date,Price
0,PIPES,SKYLINE TRADERS,2023-06-03 10:32:53,294.85
1,WOOD,IRONHOUSE,2023-08-08 06:00:48,272.98
2,WOOD,GREENFIELD LTD,2023-05-31 03:00:37,47.59
3,BRICKS,GOLDENHAMMER,2023-09-15 03:40:02,33.23
4,WOOD,IRONHOUSE,2023-04-12 23:42:14,57.21


In [75]:
# Exporting cleaned dataset into an Excel file.
df.to_excel(r"C:\Users\Windows 11\OneDrive\Desktop\Projects\Volunteer_Project\sample_dataset_cleaned.xlsx", index = False)

# price_compare & cheapest_offers

### In this file I gathered the suppliers and their products that were the cheapest.
### Also, I showed how much the company will save in average if they buy the cheapest products only.

---

In [102]:
# Importing pandas as uploading the data.
import pandas as pd
df = pd.read_excel(r"C:\Users\Windows 11\OneDrive\Desktop\Projects\Volunteer_Project\Results\sample_dataset_cleaned.xlsx")
df.head(5)

,Product,Supplier,Date,Price
0,PIPES,SKYLINE TRADERS,2023-06-03 10:32:53,294.85
1,WOOD,IRONHOUSE,2023-08-08 06:00:48,272.98
2,WOOD,GREENFIELD LTD,2023-05-31 03:00:37,47.59
3,BRICKS,GOLDENHAMMER,2023-09-15 03:40:02,33.23
4,WOOD,IRONHOUSE,2023-04-12 23:42:14,57.21


In [103]:
# Grouping the prices by product. Identifying the minimum and average prices for each product.
min_prices = df.groupby("Product")["Price"].min().reset_index()
avg_prices = df.groupby("Product")["Price"].mean().reset_index()

In [104]:
# Merging min_prices and avg_prices together. Calculating savings in percentage and USD.
price_compare = avg_prices.merge(min_prices, on = "Product", suffixes = ("_avg", "_min"))
price_compare["Saving_Percent"] = ((price_compare["Price_avg"] - price_compare["Price_min"]) / price_compare["Price_avg"]) * 100
price_compare["Saving_USD"] = price_compare["Price_avg"] - price_compare["Price_min"]

In [106]:
price_compare.head()

,Product,Price_avg,Price_min,Saving_Percent,Saving_USD
0,BRICKS,250.322887,15.40,93.847946,234.922887
1,CEMENT,241.277727,5.47,97.732903,235.807727
2,GLASS,239.129541,5.30,97.783628,233.829541
3,GRAVEL,260.830962,7.99,96.936713,252.840962
4,PAINT,248.448539,15.92,93.592234,232.528539


In [107]:
# Merging the min-prices with the data frame to identify the cheapest suppliers.
cheapest_offers = pd.merge(df, min_prices, on=["Product", "Price"])

In [108]:
cheapest_offers = cheapest_offers[['Supplier', 'Product', 'Price', 'Date']]
cheapest_offers

,Supplier,Product,Price,Date
0,MEGACONSTRUCT,PAINT,15.92,2023-06-11 00:46:04
1,MEGACONSTRUCT,SAND,7.07,2023-11-01 19:31:16
2,NOVA SUPPLY,WOOD,14.97,2023-04-24 06:33:26
3,NOVA SUPPLY,PIPES,10.30,2023-04-11 08:47:13
4,IRONHOUSE,STEEL RODS,5.11,2023-08-27 12:40:06
5,ALPHA SUPPLIES,TILES,9.91,2023-06-28 11:32:39
6,IRONHOUSE,GLASS,5.30,2023-09-25 21:00:30
7,NOVA SUPPLY,GRAVEL,7.99,2023-07-04 21:16:10
8,SKYLINE TRADERS,CEMENT,5.47,2023-04-03 19:15:10
9,ALPHA SUPPLIES,BRICKS,15.40,2023-04-25 22:51:02


In [112]:
# Exporting into an Excel file.
output_path = r"C:\Users\Windows 11\OneDrive\Desktop\Projects\Volunteer_Project\Results\Cheapest_Offers_Savings.xlsx"
with pd.ExcelWriter(output_path) as writer:
    price_compare.to_excel(writer, sheet_name='Saving', index=False)
    cheapest_offers.to_excel(writer, sheet_name='Cheapest_Offers', index=False)

# Cleaning for Power BI (Monthly Max)

### This file will be used to visualize how product prices fluctuate over time. For each supplier and each product, I took the maximum price in every month.

In [117]:
# Importing pandas as uploading the data.
import pandas as pd
df = pd.read_excel(r"C:\Users\Windows 11\OneDrive\Desktop\Projects\Volunteer_Project\Results\sample_dataset_cleaned.xlsx")
df.head(5)

,Product,Supplier,Date,Price
0,PIPES,SKYLINE TRADERS,2023-06-03 10:32:53,294.85
1,WOOD,IRONHOUSE,2023-08-08 06:00:48,272.98
2,WOOD,GREENFIELD LTD,2023-05-31 03:00:37,47.59
3,BRICKS,GOLDENHAMMER,2023-09-15 03:40:02,33.23
4,WOOD,IRONHOUSE,2023-04-12 23:42:14,57.21


In [118]:
# Turning the Date column into short date.
df['Date_str'] = df['Date'].astype(str)
df['Date_Short'] = df['Date_str'].str[:10]
df['Date_Short'] = pd.to_datetime(df['Date_Short'])

df.drop(columns=['Date_str', 'Date'], inplace=True)
df.rename(columns={'Date_Short': 'Date'}, inplace = True)

df.head(5)

,Product,Supplier,Price,Date
0,PIPES,SKYLINE TRADERS,294.85,2023-06-03
1,WOOD,IRONHOUSE,272.98,2023-08-08
2,WOOD,GREENFIELD LTD,47.59,2023-05-31
3,BRICKS,GOLDENHAMMER,33.23,2023-09-15
4,WOOD,IRONHOUSE,57.21,2023-04-12


In [119]:
# Breaking the dates down into months
df["Year_Month"] = df["Date"].dt.to_period("M").astype(str)
df["Year_Month"].head()

0    2023-06
1    2023-08
2    2023-05
3    2023-09
4    2023-04
Name: Year_Month, dtype: object

In [120]:
df.head()

,Product,Supplier,Price,Date,Year_Month
0,PIPES,SKYLINE TRADERS,294.85,2023-06-03,2023-06
1,WOOD,IRONHOUSE,272.98,2023-08-08,2023-08
2,WOOD,GREENFIELD LTD,47.59,2023-05-31,2023-05
3,BRICKS,GOLDENHAMMER,33.23,2023-09-15,2023-09
4,WOOD,IRONHOUSE,57.21,2023-04-12,2023-04


In [121]:
# Getting the maximum price of each product for each month
monthly_max = (
    df.groupby(["Product", "Year_Month"])["Price"]
    .max()
    .reset_index()
)
monthly_max.head()

,Product,Year_Month,Price
0,BRICKS,2023-01,488.40
1,BRICKS,2023-02,466.62
2,BRICKS,2023-03,473.15
3,BRICKS,2023-04,493.51
4,BRICKS,2023-05,365.42


In [124]:
# Merging monthly_max with the data frame to include the suppliers.
monthly_max_with_supplier = pd.merge(
    monthly_max,
    df,
    on=['Product', 'Year_Month', 'Price'],
    how="left"
)

In [126]:
monthly_max_with_supplier.head()

,Product,Year_Month,Price,Supplier,Date
0,BRICKS,2023-01,488.40,GREENFIELD LTD,2023-01-03
1,BRICKS,2023-02,466.62,STONEWORKS,2023-02-20
2,BRICKS,2023-03,473.15,GOLDENHAMMER,2023-03-13
3,BRICKS,2023-04,493.51,GREENFIELD LTD,2023-04-30
4,BRICKS,2023-05,365.42,SKYLINE TRADERS,2023-05-26


In [127]:
monthly_max_with_supplier = monthly_max_with_supplier[['Supplier', 'Product', 'Date', 'Price']]

In [129]:
monthly_max_with_supplier.head(10)

,Supplier,Product,Date,Price
0,GREENFIELD LTD,BRICKS,2023-01-03,488.40
1,STONEWORKS,BRICKS,2023-02-20,466.62
2,GOLDENHAMMER,BRICKS,2023-03-13,473.15
3,GREENFIELD LTD,BRICKS,2023-04-30,493.51
4,SKYLINE TRADERS,BRICKS,2023-05-26,365.42
5,GREENFIELD LTD,BRICKS,2023-06-14,483.31
6,GOLDENHAMMER,BRICKS,2023-07-22,458.66
7,SKYLINE TRADERS,BRICKS,2023-08-03,449.01
8,GOLDENHAMMER,BRICKS,2023-09-06,247.44
9,GOLDENHAMMER,BRICKS,2023-10-29,492.40


In [130]:
monthly_max_with_supplier.to_excel(r"C:\Users\Windows 11\OneDrive\Desktop\Projects\Volunteer_Project\Power_BI\Monthly_Max.xlsx", index = False)

# Price Increased or Decreased

### The company asked to seperate those who increased or decreased their prices.

In [49]:
import pandas as pd
df = pd.read_excel(r"C:\Users\Windows 11\OneDrive\Desktop\Projects\Volunteer_Project\Results\sample_dataset_cleaned.xlsx")
df.head()

,Product,Supplier,Date,Price
0,PIPES,SKYLINE TRADERS,2023-06-03 10:32:53,294.85
1,WOOD,IRONHOUSE,2023-08-08 06:00:48,272.98
2,WOOD,GREENFIELD LTD,2023-05-31 03:00:37,47.59
3,BRICKS,GOLDENHAMMER,2023-09-15 03:40:02,33.23
4,WOOD,IRONHOUSE,2023-04-12 23:42:14,57.21


In [50]:
df["Date_str"] = df["Date"].astype(str)
df["Date_str"] = df["Date_str"].str[:10] 
df["Date_Short"] = pd.to_datetime(df["Date_str"])

df.drop(columns = {"Date_str", "Date"}, inplace = True)
df.rename(columns = {"Date_Short": "Date"}, inplace = True)

In [51]:
df.head()

,Product,Supplier,Price,Date
0,PIPES,SKYLINE TRADERS,294.85,2023-06-03
1,WOOD,IRONHOUSE,272.98,2023-08-08
2,WOOD,GREENFIELD LTD,47.59,2023-05-31
3,BRICKS,GOLDENHAMMER,33.23,2023-09-15
4,WOOD,IRONHOUSE,57.21,2023-04-12


In [52]:
df.sort_values(["Product", "Supplier", "Date"], inplace = True)
df.head()

,Product,Supplier,Price,Date
330,BRICKS,ALPHA SUPPLIES,45.38,2023-01-18
928,BRICKS,ALPHA SUPPLIES,343.14,2023-02-18
541,BRICKS,ALPHA SUPPLIES,224.87,2023-03-05
903,BRICKS,ALPHA SUPPLIES,351.93,2023-04-08
406,BRICKS,ALPHA SUPPLIES,128.03,2023-04-21


In [53]:
summary = (
    df.groupby(["Product", "Supplier"])
    .agg(
        first_date = ("Date", "first"),
        last_date = ("Date", "last"),
        first_price = ("Price", "first"),
        last_price = ("Price", "last"),
    )
    .reset_index()
)

In [54]:
summary.head()

,Product,Supplier,first_date,last_date,first_price,last_price
0,BRICKS,ALPHA SUPPLIES,2023-01-18,2023-12-09,45.38,378.51
1,BRICKS,BUILDCO,2023-05-05,2023-11-24,199.69,32.09
2,BRICKS,GOLDENHAMMER,2023-02-06,2023-12-23,88.07,388.88
3,BRICKS,GREENFIELD LTD,2023-01-03,2023-12-25,488.40,201.17
4,BRICKS,IRONHOUSE,2023-02-08,2023-11-16,455.08,323.18


In [55]:
import numpy as np

summary["Price_Status"] = np.where(
    summary["last_price"] > summary["first_price"], 1,
    np.where(summary["first_price"] > summary["last_price"], 0, np.nan)
)

In [56]:
summary["Price_Status"].value_counts()

Price_Status
1.0    53
0.0    47
Name: count, dtype: int64

In [57]:
price_increase_decrease = pd.merge(
    summary,
    df,
    on = ["Product", "Supplier"],
    how = "left"
)
price_increase_decrease

,Product,Supplier,first_date,last_date,first_price,last_price,Price_Status,Price,Date
0,BRICKS,ALPHA SUPPLIES,2023-01-18,2023-12-09,45.38,378.51,1.0,45.38,2023-01-18
1,BRICKS,ALPHA SUPPLIES,2023-01-18,2023-12-09,45.38,378.51,1.0,343.14,2023-02-18
2,BRICKS,ALPHA SUPPLIES,2023-01-18,2023-12-09,45.38,378.51,1.0,224.87,2023-03-05
3,BRICKS,ALPHA SUPPLIES,2023-01-18,2023-12-09,45.38,378.51,1.0,351.93,2023-04-08
4,BRICKS,ALPHA SUPPLIES,2023-01-18,2023-12-09,45.38,378.51,1.0,128.03,2023-04-21
...,...,...,...,...,...,...,...,...,...
991,WOOD,URBAN MATERIALS,2023-01-01,2023-11-30,483.09,375.74,0.0,235.76,2023-07-17
992,WOOD,URBAN MATERIALS,2023-01-01,2023-11-30,483.09,375.74,0.0,244.23,2023-07-18
993,WOOD,URBAN MATERIALS,2023-01-01,2023-11-30,483.09,375.74,0.0,271.49,2023-07-21
994,WOOD,URBAN MATERIALS,2023-01-01,2023-11-30,483.09,375.74,0.0,380.10,2023-11-15


In [58]:
price_increase_decrease.drop(columns = {"first_date", "last_date", "first_price", "last_price"}, inplace = True)
price_increase_decrease.head()

,Product,Supplier,Price_Status,Price,Date
0,BRICKS,ALPHA SUPPLIES,1.0,45.38,2023-01-18
1,BRICKS,ALPHA SUPPLIES,1.0,343.14,2023-02-18
2,BRICKS,ALPHA SUPPLIES,1.0,224.87,2023-03-05
3,BRICKS,ALPHA SUPPLIES,1.0,351.93,2023-04-08
4,BRICKS,ALPHA SUPPLIES,1.0,128.03,2023-04-21


In [59]:
price_increase_decrease["Price_Status"] = price_increase_decrease["Price_Status"].map({
    1: "Increas",
    0: "Decrease",
    np.nan: "No Change"
})
price_increase_decrease.head()

,Product,Supplier,Price_Status,Price,Date
0,BRICKS,ALPHA SUPPLIES,Increas,45.38,2023-01-18
1,BRICKS,ALPHA SUPPLIES,Increas,343.14,2023-02-18
2,BRICKS,ALPHA SUPPLIES,Increas,224.87,2023-03-05
3,BRICKS,ALPHA SUPPLIES,Increas,351.93,2023-04-08
4,BRICKS,ALPHA SUPPLIES,Increas,128.03,2023-04-21


In [60]:
price_increase_decrease.to_excel(r"C:\Users\Windows 11\OneDrive\Desktop\Projects\Volunteer_Project\Power_BI\Price_Increase_Decrease.xlsx", index = False)
print("File is ready.")

File is ready.
